In [1]:
import numpy as np
import pandas as pd
import os
import scipy
import mne 

from mne_bids import write_raw_bids, BIDSPath, make_dataset_description

# Data available and basic information of dataset
The dataset is publicly available at https://doi.org/10.6084/m9.figshare.c.5006684 and consists of two subsets:

- **Adult Subjects**: EEG and behavioral data from 20 adult participants, labeled as Axxx (e.g., A001 for subject 1).

- **Child Subjects**: Data from 96 child participants, labeled as Cxxx.

This notebook processes the adult subset (Axxx) for conversion into the Brain Imaging Data Structure (BIDS) format.


# Features of this published dataset
The dataset, stored in '.mat' files, has been preprocessed by the authors to include:

- **Quality Control**: Trials flagged as erroneous were marked using the GoodTrial variable (0 for bad, 1 for good).
- **Preprocessing Steps**: Techniques such as Reliable Components Analysis (RCA) were applied. For details, see the authors' scripts at https://osf.io/fkjt6/.
- **Data Structure**: The .mat files use a nested structure, which we unpack using scipy.io.loadmat and indexing to extract data as NumPy ndarray objects.
- **Variables**:
    - **Core Variables**: EEG signals and event triggers (e.g., stimulus onsets, responses) for further analysis.
    - **Intermediate Variables**: Metrics like electrooculogram (EOG) data and bad electrode flags from the authors' preprocessing.



# Preprocessing Rationale
The '.mat' format is not directly compatible with MNE-Python. Converting the dataset to BIDS format ensures compatibility, enhances reproducibility, and facilitates integration with other neuroimaging tools. This notebook focuses on **transforming the raw data into BIDS format**.



# Preprocessing Workflow
The preprocessing pipeline is divided into two main steps:

## 1. Path Configuration
- **Input Directory**: Define ***ds_root*** as the location of raw '.mat' files to streamline data loading.
- **Filename Structure**: Use ***fn_first*** and ***fn_last*** to construct subject-specific filenames (e.g., Formatted_A001.3_40_preproc.mat).
- **Output Directory**: Create a ***preprocessData*** folder to store BIDS-compliant output files if it does not exist.

## 2. Conversion to BIDS Format
For each subject (A001 to A020):

1.**Behavioral Data Filtering**:
- Load behavioral data from ***behaviorData_low_1_raw.csv*** and filter for the current subject.
- Adjust trial indices (subtract 1) to align with zero-based Python indexing.
- Retain trials marked as valid (***GoodTrial == 1***) and those with non-missing response triggers, as subsequent analyses are response-locked.

2.**EEG Data Preparation**:
- Load EEG data from '.mat' files and concatenate trial segments into a 2D array of shape (n_channels, n_timepoints).
- Convert EEG amplitudes from volts to microvolts (µV) by multiplying by 1e6, per BIDS standards.

3.**Event Trigger Processing**:
- Extract triggers and event triggers (e.g., fixation, stimulus, response) from the '.mat' file.
- Filter triggers to include only valid trials with response events.
- Transform per-trial triggers into a continuous timeline by adding trial start offsets.
- Format triggers into a (n_events, 3) array for BIDS:
    - Column 1: Timestamps (sample indices, adjusted for Python’s zero-based indexing).
    - Column 2: Duration (set to 0, as it is not used in this analysis).
    - Column 3: Event IDs (e.g., 1=fixation, 5=response).

4.**Channel Information**:
- No official channel map was provided. A reference paper comparing 10-20 and HydroCel systems identified channels like **Cpz (55th)**, **Cp1 (37th)**, and **Cp2 (87th)**. For simplicity, we created a custom channelName.csv labeling channels as E1, E2, etc.
- Load channel names from this CSV for MNE compatibility.

5.**BIDS Export**:
- Create an MNE ***RawArray*** object with a sampling rate of 500 Hz and EEG channel types.
- Use ***mne_bids.write_raw_bids*** to save data in BrainVision format, including events and metadata.
- Generate a dataset description using ***make_dataset_description*** to complete the BIDS structure.

6.**Behavioral Data Output**:
- Combine filtered behavioral data across subjects into a single DataFrame.
- Save as ***behaviorData_low_1_final_test.csv*** with full columns and as ***behaviorData_low_1_final.csv*** with selected columns (subj, coherence, RT, accuracy, keypress).

In [2]:
# set different paths to load and save data
ds_root='/volumes/hyijie_psy/CPP/data_low-level_1/data' # path for raw data
tgt_dir='..'
# set file name to load data 
fn_first='Formatted_'
fn_last='.3_40_preproc'

# make folders to save result
preprocessData=os.path.join(tgt_dir,'data/preprocessData')
if not os.path.exists(preprocessData):
    os.makedirs(preprocessData)


In [ ]:
# Preprocess EEG and behavioral data for BIDS conversion

# Load and filter behavioral data for adult subjects (starting with 'A')
data_behavior = pd.read_csv('../data/behaviorData_low_1_raw.csv')
data_behavior = data_behavior[data_behavior.iloc[:, 1].astype(str).str.startswith('A')]

# Initialize a list to store filtered behavioral data for all subjects
data_behavior_filter = []

# Define subject IDs (A001 to A020)
sub_dir=[f'A{i:03d}' for i in range(1,21)]

# Process each subject
for sub in sub_dir:
    # Filter behavioral data for the current subject
    data_behavior_perSubject = data_behavior[data_behavior.iloc[:,1] == sub]
    # Adjust trial indices to zero-based for Python compatibility
    data_behavior_perSubject.iloc[:, 2] = data_behavior_perSubject.iloc[:, 2] - 1
    

    # Load EEG data, triggers, and trial epoch indices from .mat file
    raw = scipy.io.loadmat(os.path.join(ds_root,fn_first+sub+fn_last+'.mat'))
    good_trials = raw['GoodTrial'][0][0][0].T # Binary array (1=good, 0=bad)
    data_EEG = raw['X'] 
    data_trigger = raw['Onsets']['corrMatrix'][0,0] # Event timestamps
    trialEpoch_id = raw['trialEpochIdx'] # Start/end indices for each trial
    
    # Step 1: Exclude bad trials based on GoodTrial
    valid_trial = good_trials[:,0] == 1
    valid_trial_index = np.where(valid_trial)[0] # Indices of good trials
    data_trigger_goodTrial = data_trigger[valid_trial_index,:]
    trialEpoch_id_goodTrial = trialEpoch_id[valid_trial_index,:]

    # Filter behavioral data to match valid trials
    data_behavior_perSubject_goodTrial = data_behavior_perSubject[data_behavior_perSubject.iloc[:, 2].isin(valid_trial_index)]
    data_behavior_filter.append(data_behavior_perSubject_goodTrial)

    # Step 2: Exclude trials missing response triggers (response-locked analysis)
    valid_row = ~np.isnan(data_trigger_goodTrial[:,4]) # Check for non-NaN responses
    index_validTrial = np.where(valid_row)[0]
    data_trigger_dropNaN = data_trigger_goodTrial[index_validTrial,:]
    trialEpoch_id_dropNaN = trialEpoch_id_goodTrial[index_validTrial,:]

    # Prepare EEG data for BIDS: Concatenate trials and convert to microvolts
    data_EEG_BIDS = np.concatenate(data_EEG[0], axis=1)
    data_EEG_BIDS = data_EEG_BIDS * 1e6 # Shape: (n_channels, n_timepoints)
    
    # Transform triggers to continuous format for BIDS
    data_trigger_continious = np.empty((data_trigger_dropNaN.shape[0],
                                        data_trigger_dropNaN.shape[1]),
                                        dtype = object)
    for trial in range(0,data_trigger_dropNaN.shape[0]): 
        # Adjust trigger timestamps by trial start offset (subtract 1 for MATLAB-to-Python indexing)
        data_trigger_continious[trial,:] = data_trigger_dropNaN[trial,:]+trialEpoch_id_dropNaN[trial,0]-1 
    
    # Reshape triggers into BIDS format: (n_events, 3)
    trigger_row = data_trigger_continious.shape[0]
    trigger_col = data_trigger_continious.shape[1]
    data_trigger_BIDS = np.empty((trigger_row * trigger_col , 3), # must be 3 for BIDS
                                 dtype = np.int32) # note that int32 is necessary
    for trial in range(0,trigger_row): 
        # Column 1: Timestamps (adjusted for zero-based indexing)
        data_trigger_BIDS[trial*trigger_col : (trial+1)*trigger_col , 0] = data_trigger_continious[trial,:]-1 

        # Column 2: Duration (unused, set to 0)
        data_trigger_BIDS[trial*trigger_col : (trial+1)*trigger_col , 1] = 0

        # Column 3: Event IDs (1 to n_event_types)
        data_trigger_BIDS[trial*trigger_col : (trial+1)*trigger_col , 2] = np.arange(1,trigger_col+1)

    # Define sampling rate
    samplingRate = 500
    
    # Load channel names from custom CSV (e.g., E1, E2, ...)
    channel_info = pd.read_csv('../data/channelName.csv')
    channel_info = channel_info.dropna()
    channelName = channel_info['channel'].tolist()
    
    # Create MNE RawArray object for EEG data
    info = mne.create_info(ch_names=channelName, sfreq=samplingRate, ch_types='eeg')
    raw_mne = mne.io.RawArray(data_EEG_BIDS, info)

    # Define BIDS path for the current subject
    bids_path = BIDSPath(
        subject=sub, 
        task='randomDot', 
        datatype='eeg',
        root=preprocessData
    )

    # Export to BIDS format (BrainVision) with event annotations
    write_raw_bids(
        raw=raw_mne,
        bids_path=bids_path,
        events=data_trigger_BIDS,
        event_id={'fixation':1,
                  'boil':2,
                  'photodiode':3,
                  'stimulus':4, # Note: Timings corrected relative to photodiode signal
                  'response':5, 
                  'offser':6}, 
        allow_preload=True,
        format='BrainVision',
        overwrite=True,
        verbose=True
    )

    # Generate BIDS dataset description
    make_dataset_description(
    path=preprocessData,
    name='LowLevel-randomDot',
    dataset_type='raw',
    overwrite=True
    )

# Combine and save filtered behavioral data
data_behavior_filter_final = np.concatenate(data_behavior_filter, axis=0)
data_behavior_filter_final = pd.DataFrame(data_behavior_filter_final)
data_behavior_filter_final.columns = ['Pnumber', 'subj', 'trialno','group', 'age', 'sex', 'hands', 'devcond', 'coherence', 'motiondir', 'keypress', 'RT', 'accuracy'] 

# Save selected behavioral columns for analysis
selected_columns = ['subj','coherence', 'RT', 'accuracy', 'keypress']
data_behavior_selected = data_behavior_filter_final[selected_columns]
data_behavior_selected.to_csv('../data/behaviorData_low_1_final.csv', index=False, header=True)



Creating RawArray with float64 data, n_channels=128, n_times=491753
    Range : 0 ... 491752 =      0.000 ...   983.504 secs
Ready.
Writing '../data/preprocessData/participants.tsv'...
Writing '../data/preprocessData/participants.json'...
Used Annotations descriptions: [np.str_('boil'), np.str_('fixation'), np.str_('offser'), np.str_('photodiode'), np.str_('response'), np.str_('stimulus')]
Writing '../data/preprocessData/sub-A001/eeg/sub-A001_task-randomDot_events.tsv'...
Writing '../data/preprocessData/sub-A001/eeg/sub-A001_task-randomDot_events.json'...
Writing '../data/preprocessData/dataset_description.json'...
Writing '../data/preprocessData/sub-A001/eeg/sub-A001_task-randomDot_eeg.json'...
Found no extension for raw file, assuming "BTi" format and appending extension .pdf
Writing '../data/preprocessData/sub-A001/eeg/sub-A001_task-randomDot_channels.tsv'...
Copying data files to sub-A001_task-randomDot_eeg.vhdr


/var/folders/xc/16fqwx1n1gbfb77yd1bdg8mh0000gn/T/ipykernel_3337/2620168588.py:84: RuntimeWarning: Converting data files to BrainVision format
  write_raw_bids(
/var/folders/xc/16fqwx1n1gbfb77yd1bdg8mh0000gn/T/ipykernel_3337/2620168588.py:84: RuntimeWarning: Encountered data in "double" format. Converting to float32.
  write_raw_bids(


Writing '../data/preprocessData/sub-A001/sub-A001_scans.tsv'...
Wrote ../data/preprocessData/sub-A001/sub-A001_scans.tsv entry with eeg/sub-A001_task-randomDot_eeg.vhdr.
Writing '../data/preprocessData/dataset_description.json'...
Creating RawArray with float64 data, n_channels=128, n_times=461226
    Range : 0 ... 461225 =      0.000 ...   922.450 secs
Ready.
Writing '../data/preprocessData/participants.tsv'...
Writing '../data/preprocessData/participants.json'...
Used Annotations descriptions: [np.str_('boil'), np.str_('fixation'), np.str_('offser'), np.str_('photodiode'), np.str_('response'), np.str_('stimulus')]
Writing '../data/preprocessData/sub-A002/eeg/sub-A002_task-randomDot_events.tsv'...
Writing '../data/preprocessData/sub-A002/eeg/sub-A002_task-randomDot_events.json'...
Writing '../data/preprocessData/dataset_description.json'...
Writing '../data/preprocessData/sub-A002/eeg/sub-A002_task-randomDot_eeg.json'...
Found no extension for raw file, assuming "BTi" format and appen

/var/folders/xc/16fqwx1n1gbfb77yd1bdg8mh0000gn/T/ipykernel_3337/2620168588.py:84: RuntimeWarning: Converting data files to BrainVision format
  write_raw_bids(
/var/folders/xc/16fqwx1n1gbfb77yd1bdg8mh0000gn/T/ipykernel_3337/2620168588.py:84: RuntimeWarning: Encountered data in "double" format. Converting to float32.
  write_raw_bids(


Writing '../data/preprocessData/sub-A002/sub-A002_scans.tsv'...
Wrote ../data/preprocessData/sub-A002/sub-A002_scans.tsv entry with eeg/sub-A002_task-randomDot_eeg.vhdr.
Writing '../data/preprocessData/dataset_description.json'...
Creating RawArray with float64 data, n_channels=128, n_times=449384
    Range : 0 ... 449383 =      0.000 ...   898.766 secs
Ready.
Writing '../data/preprocessData/participants.tsv'...
Writing '../data/preprocessData/participants.json'...
Used Annotations descriptions: [np.str_('boil'), np.str_('fixation'), np.str_('offser'), np.str_('photodiode'), np.str_('response'), np.str_('stimulus')]
Writing '../data/preprocessData/sub-A003/eeg/sub-A003_task-randomDot_events.tsv'...
Writing '../data/preprocessData/sub-A003/eeg/sub-A003_task-randomDot_events.json'...
Writing '../data/preprocessData/dataset_description.json'...
Writing '../data/preprocessData/sub-A003/eeg/sub-A003_task-randomDot_eeg.json'...
Found no extension for raw file, assuming "BTi" format and appen

...

Writing '../data/preprocessData/sub-A018/sub-A018_scans.tsv'...
Wrote ../data/preprocessData/sub-A018/sub-A018_scans.tsv entry with eeg/sub-A018_task-randomDot_eeg.vhdr.
Writing '../data/preprocessData/dataset_description.json'...
Creating RawArray with float64 data, n_channels=128, n_times=455674
    Range : 0 ... 455673 =      0.000 ...   911.346 secs
Ready.
Writing '../data/preprocessData/participants.tsv'...
Writing '../data/preprocessData/participants.json'...
Used Annotations descriptions: [np.str_('boil'), np.str_('fixation'), np.str_('offser'), np.str_('photodiode'), np.str_('response'), np.str_('stimulus')]
Writing '../data/preprocessData/sub-A019/eeg/sub-A019_task-randomDot_events.tsv'...
Writing '../data/preprocessData/sub-A019/eeg/sub-A019_task-randomDot_events.json'...
Writing '../data/preprocessData/dataset_description.json'...
Writing '../data/preprocessData/sub-A019/eeg/sub-A019_task-randomDot_eeg.json'...
Found no extension for raw file, assuming "BTi" format and appen

/var/folders/xc/16fqwx1n1gbfb77yd1bdg8mh0000gn/T/ipykernel_3337/2620168588.py:84: RuntimeWarning: Converting data files to BrainVision format
  write_raw_bids(
/var/folders/xc/16fqwx1n1gbfb77yd1bdg8mh0000gn/T/ipykernel_3337/2620168588.py:84: RuntimeWarning: Encountered data in "double" format. Converting to float32.
  write_raw_bids(


Writing '../data/preprocessData/sub-A019/sub-A019_scans.tsv'...
Wrote ../data/preprocessData/sub-A019/sub-A019_scans.tsv entry with eeg/sub-A019_task-randomDot_eeg.vhdr.
Writing '../data/preprocessData/dataset_description.json'...
Creating RawArray with float64 data, n_channels=128, n_times=432717
    Range : 0 ... 432716 =      0.000 ...   865.432 secs
Ready.
Writing '../data/preprocessData/participants.tsv'...
Writing '../data/preprocessData/participants.json'...
Used Annotations descriptions: [np.str_('boil'), np.str_('fixation'), np.str_('offser'), np.str_('photodiode'), np.str_('response'), np.str_('stimulus')]
Writing '../data/preprocessData/sub-A020/eeg/sub-A020_task-randomDot_events.tsv'...
Writing '../data/preprocessData/sub-A020/eeg/sub-A020_task-randomDot_events.json'...
Writing '../data/preprocessData/dataset_description.json'...
Writing '../data/preprocessData/sub-A020/eeg/sub-A020_task-randomDot_eeg.json'...
Found no extension for raw file, assuming "BTi" format and appen

/var/folders/xc/16fqwx1n1gbfb77yd1bdg8mh0000gn/T/ipykernel_3337/2620168588.py:84: RuntimeWarning: Converting data files to BrainVision format
  write_raw_bids(
/var/folders/xc/16fqwx1n1gbfb77yd1bdg8mh0000gn/T/ipykernel_3337/2620168588.py:84: RuntimeWarning: Encountered data in "double" format. Converting to float32.
  write_raw_bids(


Writing '../data/preprocessData/sub-A020/sub-A020_scans.tsv'...
Wrote ../data/preprocessData/sub-A020/sub-A020_scans.tsv entry with eeg/sub-A020_task-randomDot_eeg.vhdr.
Writing '../data/preprocessData/dataset_description.json'...
